In [1]:
import numpy as np

In [13]:
# data = np.array([[.75,2000,18,50,500],
#              [.5,1500,20,40,450],
#              [.9,2050,25,35,800]])


data = np.array([
            [4,4,3,5,3],
            [3,2,3,5,2],
            [4.5,5,2,2,3],
            [3.5,5,2,2,3],
            [3,2,3,5,2]
                ])


# Normalize the decision matrix
normalized_data = data / np.sqrt(np.sum(data**2,axis=0))
normalized_data

array([[0.49051147, 0.46499055, 0.50709255, 0.5488213 , 0.50709255],
       [0.3678836 , 0.23249528, 0.50709255, 0.5488213 , 0.3380617 ],
       [0.55182541, 0.58123819, 0.3380617 , 0.21952852, 0.50709255],
       [0.42919754, 0.58123819, 0.3380617 , 0.21952852, 0.50709255],
       [0.3678836 , 0.23249528, 0.50709255, 0.5488213 , 0.3380617 ]])

In [15]:
weights = np.array([5,4,4,5,3])
weighted = normalized_data * weights
weighted

array([[2.45255736, 1.85996222, 2.02837021, 2.7441065 , 1.52127766],
       [1.83941802, 0.92998111, 2.02837021, 2.7441065 , 1.01418511],
       [2.75912703, 2.32495277, 1.35224681, 1.0976426 , 1.52127766],
       [2.14598769, 2.32495277, 1.35224681, 1.0976426 , 1.52127766],
       [1.83941802, 0.92998111, 2.02837021, 2.7441065 , 1.01418511]])

In [16]:
dec_matrix = np.array([False,True,True,True,True])

maximizing = np.where(dec_matrix,np.max(weighted,axis=0),np.min(weighted,axis=0))
maximizing

array([1.83941802, 2.32495277, 2.02837021, 2.7441065 , 1.52127766])

In [17]:
minimizing = np.where(dec_matrix,np.min(weighted,axis=0),np.max(weighted,axis=0))
minimizing

array([2.75912703, 0.92998111, 1.35224681, 1.0976426 , 1.01418511])

In [19]:
distance_to_ideal = np.sqrt(np.sum((normalized_data - maximizing)**2, axis=1))
# distance_to_worst = np.sqrt(np.sum((matrix - minimizing)**2, axis=1))

distance_to_ideal

array([3.66621639, 3.88295696, 3.86751128, 3.91004739, 3.88295696])

In [38]:
distance_to_worst = np.sqrt(np.sum((normalized_data - minimizing)**2, axis=1))

distance_to_worst,distance_to_worst[::]

(array([2.57593986, 2.77076975, 2.65529672, 2.75807753, 2.77076975]),
 array([2.57593986, 2.77076975]))

In [29]:
relative_closeness = distance_to_worst / (distance_to_ideal + distance_to_worst)

# Rank the alternatives
ranked_alternatives = np.argsort(relative_closeness)[::-1]
ranked_alternatives.reshape([5,1] )

array([[4],
       [1],
       [3],
       [0],
       [2]])